In [1]:
# @title Step 0: Setup and Installation
# Install ADK and LiteLLM for multi-model support


!pip install google-adk -q
!pip install litellm -q

print("Installation complete.")

Installation complete.


In [2]:
# @title Import necessary libraries
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

Libraries imported.


In [ ]:
# @title Configure API Keys (Load from .env file)

# Install python-dotenv if not already installed
!pip install python-dotenv -q

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Gemini API Key (loaded from .env)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY if GOOGLE_API_KEY else ""

# [Optional]
# OpenAI API Key (Get from OpenAI Platform: https://platform.openai.com/api-keys)
# OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
# os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY if OPENAI_API_KEY else ""

# [Optional]
# Anthropic API Key (Get from Anthropic Console: https://console.anthropic.com/settings/keys)
# ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
# os.environ['ANTHROPIC_API_KEY'] = ANTHROPIC_API_KEY if ANTHROPIC_API_KEY else ""

# --- Verify Keys (Optional Check) ---
print("API Keys Set:")
print(f"Google API Key set: {'Yes' if GOOGLE_API_KEY and GOOGLE_API_KEY != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
# print(f"OpenAI API Key set: {'Yes' if OPENAI_API_KEY and OPENAI_API_KEY != 'YOUR_OPENAI_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
# print(f"Anthropic API Key set: {'Yes' if ANTHROPIC_API_KEY and ANTHROPIC_API_KEY != 'YOUR_ANTHROPIC_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

# Configure ADK to use API keys directly (not Vertex AI for this multi-model setup)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# @markdown **Security Note:** It's best practice to manage API keys securely (e.g., using Colab Secrets or environment variables) rather than hardcoding them directly in the notebook. Now loading from .env file.